##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 케라스를 사용한 다중 워커(Multi-worker) 훈련

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/multi_worker_with_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/distribute/multi_worker_with_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/distribute/multi_worker_with_keras.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>
  </td>
</table>

## 개요

이 튜토리얼에서는 `tf.distribute.Strategy` API, 특히 `tf.distribute.MultiWorkerMirroredStrategy` 클래스를 사용하여 Keras 모델과 `Model.fit` API로 다중 작업자 분산 훈련을 수행하는 방법을 보여줍니다. 이 전략의 도움으로 단일 작업자에서 실행되도록 설계된 Keras 모델이 최소한의 코드 변경만으로 여러 작업자에서 원활하게 작동할 수 있습니다.

`tf.distribute.Strategy` API에 대해 더 깊이 이해하고 싶으신 분들은 [TensorFlow의 분산 훈련](../../guide/distributed_training.ipynb) 가이드에서 TensorFlow가 지원하는 분배 전략의 개요를 알아볼 수 있습니다.

Keras 및 사용자 정의 훈련 루프와 함께 `MultiWorkerMirroredStrategy`를 사용하는 방법에 대해 알아보려면 [Keras 및 MultiWorkerMirroredStrategy를 이용한 사용자 정의 훈련 루프](multi_worker_with_ctl.ipynb)를 참조하세요.

이 튜토리얼의 목적은 두 개의 작업자가 있는 최소 다중 작업자 예제를 보여주는 것입니다.

## 설정

몇 가지 필요한 가져오기로 시작합니다.

In [ ]:
import json
import os
import sys

TensorFlow를 가져오기 전에 환경을 몇 가지 변경합니다.

1. 모든 GPU를 비활성화합니다. 이렇게 하면 작업자가 모두 동일한 GPU를 사용하려고 하여 발생하는 오류를 방지할 수 있습니다. 실제 애플리케이션에서는 각 작업자가 다른 시스템에 있게 될 것입니다.

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

1. `TF_CONFIG` 환경 변수를 재설정합니다(나중에 자세히 알아볼 것입니다).

In [ ]:
os.environ.pop('TF_CONFIG', None)

1. 현재 디렉터리가 Python의 경로에 있도록 합니다. 그러면 나중에 `%%writefile`이 쓴 파일을 노트북이 가져올 수 있습니다.


In [ ]:
if '.' not in sys.path:
  sys.path.insert(0, '.')

이제 TensorFlow를 가져옵니다.

In [ ]:
import tensorflow as tf

### 데이터세트 및 모델 정의

Next, create an `mnist_setup.py` file with a simple model and dataset setup. This Python file will be used by the worker processes in this tutorial:

In [ ]:
def mnist_dataset(batch_size):
  (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
  # `x` 배열의 값은 uint8이고 [0, 255] 범위입니다.
  # 이를 [0, 1] 사이의 float32 값으로 바꾸어야 합니다.
  x_train = x_train / np.float32(255)
  y_train = y_train.astype(np.int64)
  train_dataset = tf.data.Dataset.from_tensor_slices(
      (x_train, y_train)).shuffle(60000).repeat().batch(batch_size)
  return train_dataset

### 단일 작업자에 대한 모델 훈련

먼저 몇 개의 epoch에 대해 모델을 훈련하고 *단일 작업자*의 결과를 관찰하여 이상이 없는지 확인합니다. 훈련이 진행됨에 따라 손실은 감소하고 정확도는 증가합니다.

In [ ]:
per_worker_batch_size = 64
single_worker_dataset = mnist_dataset(per_worker_batch_size)
single_worker_model = build_and_compile_cnn_model()
single_worker_model.fit(single_worker_dataset, epochs=3, steps_per_epoch=70)

## Multi-worker configuration

이제 다중 작업자 훈련의 세계로 들어가 보겠습니다.

### 작업 및 태스크가 있는 클러스터

TensorFlow에서 분산 훈련에는 여러 작업이 있는 `'cluster'`가 포함되며 각 작업에는 하나 이상의 `'task'`가 있을 수 있습니다.

각각의 역할이 다를 수 있는 여러 머신에서의 훈련을 위해 `TF_CONFIG` 구성 환경 변수가 필요합니다. `TF_CONFIG`는 클러스터의 일부인 각 작업자에 대한 클러스터 구성을 지정하는 데 사용되는 JSON 문자열입니다.

`TF_CONFIG` 변수에는 `'cluster'` 및 `'task'`의 두 가지 구성요소가 있습니다.

- `'cluster'`는 모든 작업자에 대해 동일하며 `'worker'` 또는 `'chief'`와 같은 다양한 유형의 작업으로 구성된 사전인 훈련 클러스터에 대한 정보를 제공합니다.

    - `tf.distribute.MultiWorkerMirroredStrategy`를 이용한 다중 작업자 훈련의 경우, 일반적으로 하나의 `'worker'`가 있으며, 이 작업자가 정규 `'worker'`가 수행하는 작업과 더불어 체크포인트를 저장하고 TensorBoard에 대한 요약 파일을 작성하는 등의 작업을 책임집니다. 이러한 `'worker'`를 책임 작업자라고 합니다(작업 이름은 `'chief'`).
    - `'chief'`가 `0`이 지정된 `'index'`를 갖는 것이 관례적입니다(실제로, 이것이 `tf.distribute.Strategy`가 구현되는 방식임).

- `'task'`는 현재 작업에 대한 정보를 제공하며 작업자마다 다릅니다. 이를 통해 해당 작업자의 `'type'`과 `'index'`가 지정됩니다.

다음은 구성의 예입니다.

In [ ]:
tf_config = {
    'cluster': {
        'worker': ['localhost:12345', 'localhost:23456']
    },
    'task': {'type': 'worker', 'index': 0}
}

다음은 JSON 문자열로 직렬화된 동일한 `TF_CONFIG`입니다.

In [ ]:
json.dumps(tf_config)

`tf_config`는 Python의 지역 변수일 뿐입니다. 훈련 구성에 사용할 수 있으려면 이 사전을 JSON으로 직렬화하고 `TF_CONFIG` 환경 변수에 배치해야 합니다.

위의 구성 예에서 `'type'` 작업을 `'worker'`로 설정하고 `'index'` 작업을 `0`으로 설정합니다. 따라서 이 시스템은 *첫 번째* 작업자입니다. 이 작업자는 `'chief'` 작업자로 지정되어 다른 작업자보다 더 많은 작업을 처리합니다.

참고: 다른 시스템에도 `TF_CONFIG` 환경 변수가 설정되어 있어야 하며 동일한 `'cluster'` 사전이 있어야 하지만 해당 시스템의 역할에 따라 다른 작업 `'type'` 또는 작업 `'index'`를 갖습니다.

설명을 위해 이 튜토리얼에서는 `localhost`에 두 작업자가 있는 `TF_CONFIG` 변수를 설정하는 방법을 보여줍니다.

실제로는 외부 IP 주소/포트에서 여러 작업자를 만들고 그에 따라 각 작업자에 `TF_CONFIG` 변수를 설정합니다.

이 튜토리얼에서는 두 개의 작업자를 사용합니다.

- 첫 번째(`'chief'`) 작업자의 `TF_CONFIG`는 위에 나와 있습니다.
- 두 번째 작업자의 경우 `tf_config['task']['index']=1`을 설정합니다.

### 노트북의 환경 변수 및 하위 프로세스

하위 프로세스는 상위로부터 환경 변수를 상속합니다.

예를 들어 이 Jupyter Notebook 프로세스에서 다음과 같이 환경 변수를 설정할 수 있습니다.

In [ ]:
os.environ['GREETINGS'] = 'Hello TensorFlow!'

그런 다음 하위 프로세스에서 환경 변수에 액세스할 수 있습니다.

In [ ]:
%%bash
echo ${GREETINGS}

다음 섹션에서는 유사한 방법을 사용하여 작업자 하위 프로세스로 `TF_CONFIG`를 전달합니다. 실제 시나리오에서 이러한 방식으로 작업을 시작하지는 않지만 이 예에서는 충분합니다.

## 적절한 전략 선택

TensorFlow에는 두 가지 주요 형태의 분산 훈련이 있습니다.

- 훈련 단계가 작업자와 복제본 간에 동기화되는 *동기식 훈련*
- 훈련 단계가 엄격하게 동기화되지 않는 *비동기 훈련*(예: [파라미터 서버 훈련](parameter_server_training.ipynb))

이 튜토리얼에서는 `tf.distribute.MultiWorkerMirroredStrategy` 인스턴스를 사용하여 동기식 다중 작업자 훈련을 수행하는 방법을 보여줍니다.

`MultiWorkerMirroredStrategy`는 모든 작업자에 걸쳐 각 장치의 모델 레이어에 있는 모든 변수의 복사본을 만듭니다. 집합체 통신을 위한 TensorFlow op인 `CollectiveOps`를 사용하여 그래디언트를 집계하고 변수를 동기화 상태로 유지합니다. 이 전략에 대한 자세한 내용은 [`tf.distribute.Strategy` 가이드](../../guide/distributed_training.ipynb)에 나와 있습니다.

In [ ]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()

Note: `TF_CONFIG` is parsed and TensorFlow's GRPC servers are started at the time `MultiWorkerMirroredStrategy` is called, so the `TF_CONFIG` environment variable must be set before a `tf.distribute.Strategy` instance is created. Since `TF_CONFIG` is not set yet, the above strategy is effectively single-worker training.

`MultiWorkerMirroredStrategy` provides multiple implementations via the `tf.distribute.experimental.CommunicationOptions` parameter: 1) `RING` implements ring-based collectives using gRPC as the cross-host communication layer; 2) `NCCL` uses the [NVIDIA Collective Communication Library](https://developer.nvidia.com/nccl) to implement collectives; and 3) `AUTO` defers the choice to the runtime. The best choice of collective implementation depends upon the number and kind of GPUs, and the network interconnect in the cluster.

## 모델 훈련하기

`tf.distribute.Strategy` API를 `tf.keras`에 통합하면 여러 작업자에게 훈련을 분배하기 위해 유일하게 수행하는 변경은 모델 구축 및 `model.compile()` 호출을 `strategy.scope()`로 감싸는 것입니다. 분산 전략의 범위에 따라 변수가 생성되는 방식과 위치가 결정되고 `MultiWorkerMirroredStrategy`의 경우, 생성된 변수는 `MirroredVariable`이며 각 작업자에 복제됩니다.


In [ ]:
with strategy.scope():
  # Model building/compiling need to be within `strategy.scope()`.
  multi_worker_model = mnist_setup.build_and_compile_cnn_model()

참고: 현재, `MultiWorkerMirroredStrategy`에는 전략 인스턴스가 생성된 후 TensorFlow ops를 생성해야 하는 제한이 있습니다. `RuntimeError: Collective ops must be configured at program startup`가 발생하면 프로그램 맨 처음에 `MultiWorkerMirroredStrategy` 인스턴스를 만들고 전략이 인스턴스화된 후에 ops를 생성할 수 있는 코드를 넣으세요.

실제로 `MultiWorkerMirroredStrategy`로 실행하려면 작업자 프로세스를 실행하고 `TF_CONFIG`를 전달해야 합니다.

Like the `mnist_setup.py` file written earlier, here is the `main.py` that each of the workers will run:

In [ ]:
NUM_WORKERS = 4

# 여기서 배치 크기는 워커의 수를 곱한 크기로 늘려야 합니다. `tf.data.Dataset.batch`에는
# 전역 배치 크기를 지정해야 하기 때문입니다. 전에는 64였지만, 이제 128이 됩니다.
global_batch_size = per_worker_batch_size * num_workers
multi_worker_dataset = mnist_dataset(global_batch_size)

with strategy.scope():
  # `strategy.scope()` 안에서 모델을 만들고 컴파일합니다.
  multi_worker_model = build_and_compile_cnn_model()

# 케라스의 `model.fit()`은 지정된 에포크 횟수와 에포크 당 스텝 수로 모델을 훈련합니다.
# 여기에 나온 숫자는 예시를 보이기 위한 것일 뿐 제대로 된 모델을 만드는데 충분하지 않습니다.
multi_worker_model.fit(multi_worker_dataset, epochs=3, steps_per_epoch=70)

위의 코드 스니펫에서 `Dataset.batch`로 전달되는 `global_batch_size`가 `per_worker_batch_size * num_workers`로 설정된다는 점에 주목하세요. 그러면 각 작업자가 작업자 수에 관계없이 `per_worker_batch_size` 예제 배치를 처리하게 됩니다.

현재 디렉터리에는 이제 두 Python 파일이 모두 포함됩니다.

In [ ]:
%%bash
ls *.py

따라서 `TF_CONFIG`를 json 직렬화하고 환경 변수에 추가합니다.

In [ ]:
os.environ['TF_CONFIG'] = json.dumps(tf_config)

이제, `main.py`를 실행하고 `TF_CONFIG`를 사용하는 작업자 프로세스를 시작할 수 있습니다.

In [ ]:
# first kill any previous runs
%killbgscripts

In [ ]:
%%bash --bg
python main.py &> job_0.log

위의 명령에서 몇 가지 주의할 사항이 있습니다.

1. 일부 bash 명령을 실행하기 위해 <a>노트북 "매직"</a>인 <code>%%bash</code>가 사용됩니다.
2. `--bg` 플래그를 사용하여 백그라운드에서 `bash` 프로세스를 실행합니다. 이 작업자는 종료되지 않기 때문입니다. 시작하기 전에 모든 작업자를 기다립니다.

백그라운드 작업자 프로세스는 이 노트북에 출력을 인쇄하지 않으므로 `&>`는 출력을 파일로 리디렉션하고 나중에 로그 파일에서 발생한 상황을 검사할 수 있습니다.

따라서 프로세스가 시작될 때까지 몇 초 동안 기다리세요.

In [ ]:
import time
time.sleep(10)

이제, 지금까지 작업자의 로그 파일에 출력된 내용을 검사합니다.

In [ ]:
%%bash
cat job_0.log

로그 파일의 마지막 줄은 다음과 같아야 합니다: `Started server with target: grpc://localhost:12345`. 이제 첫 번째 작업자가 준비되었으며 다른 모든 작업자가 계속 진행할 준비가 되기를 기다립니다.

따라서 두 번째 작업자 프로세스가 시작되도록 `tf_config`를 업데이트합니다.

In [ ]:
tf_config['task']['index'] = 1
os.environ['TF_CONFIG'] = json.dumps(tf_config)

두 번째 작업자를 시작합니다. 모든 작업자가 활성 상태이므로 훈련이 시작됩니다(따라서 이 프로세스를 백그라운드로 처리할 필요가 없음).

In [ ]:
%%bash
python main.py

첫 번째 작업자가 작성한 로그를 다시 확인하면 작업자가 해당 모델 훈련에 참여했음을 알 수 있습니다.

In [ ]:
%%bash
cat job_0.log

당연히 이 튜토리얼의 시작 부분에서 했던 테스트 실행보다 *느리게* 실행되었습니다.

단일 시스템에서 여러 작업자를 실행하면 오버헤드만 추가됩니다.

여기서 목표는 훈련 시간을 늘리는 것이 아니라 다중 작업자 훈련의 예를 제공하는 것입니다.

In [ ]:
# Delete the `TF_CONFIG`, and kill any background tasks so they don't affect the next section.
os.environ.pop('TF_CONFIG', None)
%killbgscripts

## 다중 작업자 훈련 심층 해부

지금까지 기본적인 다중 작업자 설정을 수행하는 방법을 배웠습니다.

튜토리얼의 나머지 부분에서는 실제 사용 사례에 유용하거나 중요할 수 있는 다른 요인들에 대해 자세히 알아봅니다.

### 데이터세트 샤딩

다중 작업자 훈련 중 수렴과 성능을 보장하려면 데이터 샤딩이 필요합니다.

이전 섹션의 예는 `tf.distribute.Strategy` API에서 제공하는 기본 자동 샤딩을 이용합니다. `tf.data.experimental.DistributeOptions`의 `tf.data.experimental.AutoShardPolicy`를 설정하여 샤딩을 제어할 수 있습니다.

*자동 샤딩*에 대한 자세한 내용은 [분산 입력 가이드](https://www.tensorflow.org/tutorials/distribute/input#sharding)를 참조하세요.

다음은 각 복제본이 모든 예를 처리하도록 자동 샤딩을 해제하는 방법을 보여주는 간단한 예입니다(*권장하지 않음*).


In [ ]:
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF
train_datasets_no_auto_shard = train_datasets.with_options(options)

### Evaluation

`validation_data`를 `Model.fit`으로 전달하면 각 epoch마다 훈련과 평가가 번갈아 이루어집니다. `validation_data`를 사용하는 평가는 동일한 작업자 세트 전체에 걸쳐 분배되고 평가 결과가 집계되어 모든 작업자가 사용할 수 있습니다.

훈련과 마찬가지로 검증 데이터세트는 파일 수준에서 자동으로 샤딩됩니다. 검증 데이터세트에서 전역 배치 크기를 설정하고 `validation_steps`를 설정해야 합니다.

평가를 위해 반복되는 데이터세트도 권장됩니다.

또는 주기적으로 체크포인트를 읽고 평가를 실행하는 다른 작업을 생성할 수도 있습니다. 이것이 Estimator가 하는 일입니다. 그러나 이는 권장되는 평가 방법이 아니므로 자세한 내용은 생략합니다.

### 성능

이제 `MultiWorkerMirroredStrategy`를 사용하여 여러 작업자에서 실행되도록 설정된 Keras이 있습니다.

다중 작업자 훈련의 성능을 조정하기 위해 다음을 시도할 수 있습니다.

- `tf.distribute.MultiWorkerMirroredStrategy`는 다수의 [집합 통신 구현](https://www.tensorflow.org/api_docs/python/tf/distribute/experimental/CommunicationImplementation)을 제공합니다.

    - `RING`은 호스트 사이의 통신 레이어로 gRPC를 사용하여 링 기반 집합체를 구현합니다.
    - `NCCL`은 [NVIDIA 집합 통신 라이브러리](https://developer.nvidia.com/nccl)를 사용하여 집합체를 구현합니다.
    - `AUTO`는 선택을 런타임으로 미룹니다.

    최선의 집합체 구현 선택은 클러스터의 GPU 수, GPU 유형 및 네트워크 상호 연결에 따라 다릅니다. 자동 선택을 재정의하려면 `MultiWorkerMirroredStrategy` 생성자의 `communication_options` 매개변수를 지정합니다. 예를 들면 다음과 같습니다.

    ```python
    communication_options=tf.distribute.experimental.CommunicationOptions(implementation=tf.distribute.experimental.CollectiveCommunication.NCCL)
    ```

- 가능한 경우 변수를 `tf.float`로 캐스팅합니다.

    - 공식 ResNet 모델에는 이 작업을 수행하는 방법에 대한 [예제](https://github.com/tensorflow/models/blob/8367cf6dabe11adf7628541706b660821f397dce/official/resnet/resnet_model.py#L466)가 포함되어 있습니다.

### ModelCheckpoint 콜백

동기식 훈련에서 작업자 중 하나가 실패하고 실패 복구 메커니즘이 없으면 클러스터가 실패합니다.

`tf.distribute.Strategy`와 함께 Keras를 사용하면 작업자가 작동하지 않거나 불안정한 경우 내결함성의 이점이 있습니다. 이를 위해 이전에 실패했거나 선점한 인스턴스를 다시 시작할 때 훈련 상태가 복구되도록 선택한 분산 파일 시스템에서 훈련 상태를 유지할 수 있습니다.

작업자를 사용할 수 없게 되면 다른 작업자가 실패합니다(시간 초과 후). 이러한 경우 사용할 수 없는 작업자는 물론 실패한 다른 작업자도 다시 시작해야 합니다.

Note: Previously, the `ModelCheckpoint` callback provided a mechanism to restore the training state upon a restart from a job failure for multi-worker training. The TensorFlow team are introducing a new [`BackupAndRestore`](#scrollTo=kmH8uCUhfn4w) callback, which also adds the support to single-worker training for a consistent experience, and removed the fault tolerance functionality from existing `ModelCheckpoint` callback. From now on, applications that rely on this behavior should migrate to the new `BackupAndRestore` callback.

#### ModelCheckpoint 콜백

`ModelCheckpoint` 콜백은 더 이상 내결함성 기능을 제공하지 않습니다. 대신 [`BackupAndRestore`](#scrollTo=kmH8uCUhfn4w) 콜백을 사용하세요.

`ModelCheckpoint` 콜백은 여전히 체크포인트를 저장하는 데 사용할 수 있습니다. 하지만 이를 통해 훈련이 중단되거나 성공적으로 종료된 경우 체크포인트에서 훈련을 계속하기 위해 사용자가 수동으로 모델을 로드해야 합니다.

필요한 경우, `ModelCheckpoint` 콜백 외부에서 모델/가중치를 저장하고 복원하도록 선택할 수 있습니다.

### 모델 저장 및 로드하기

`model.save` 또는 `tf.saved_model.save`를 사용하여 모델을 저장하려면 작업자마다 저장 대상이 달라야 합니다.

- 수석 작업자가 아닌 경우 모델을 임시 디렉터리에 저장해야 합니다.
- 수석 작업자의 경우 제공된 모델 디렉터리에 저장해야 합니다.

여러 작업자가 동일한 위치에 쓰려고 하여 오류가 발생하는 것을 방지하기 위해 작업자의 임시 디렉터리는 고유해야 합니다.

모든 디렉터리에 저장되는 모델은 동일하며, 일반적으로 수석 작업자가 저장한 모델만 복원 또는 제공을 위해 참조해야 합니다.

훈련이 완료되면 작업자가 만든 임시 디렉터리를 삭제하는 정리 논리가 있어야 합니다.

수석 작업자와 작업자에서 동시에 저장하는 이유는 체크포인트 동안 변수를 집계해야 할 수 있고, 이를 위해 수석 작업자와 작업자가 모두 allreduce 통신 프로토콜에 참여해야 하기 때문입니다. 반면, 수석 작업자와 작업자를 같은 모델 디렉터리에 저장하도록 하면 경합으로 인해 오류가 발생합니다.

`MultiWorkerMirroredStrategy`를 사용하면 프로그램이 모든 작업자에서 실행되고 현재 작업자가 수석 작업자인지 여부를 알기 위해 `task_type` 및 `task_id` 속성을 갖는 클러스터 확인자 개체를 이용합니다.

- `task_type`은 현재 작업이 무엇인지 알려줍니다(예: `'worker'`).
- `task_id`는 작업자의 식별자를 알려줍니다.
- `task_id == 0`인 작업자가 수석 작업자로 지정됩니다.

아래 코드 스니펫에서 `write_filepath`는 작업자의 `task_id`에 따라 쓸 파일 경로를 제공합니다.

- 수석 작업자(`task_id == 0`)의 경우 원래 파일 경로에 씁니다.
- 다른 작업자의 경우 쓸 디렉터리 경로에 `task_id`가 있는 임시 디렉터리인 `temp_dir`가 만들어집니다.

In [ ]:
model_path = '/tmp/keras-model'

def _is_chief(task_type, task_id):
  # Note: there are two possible `TF_CONFIG` configuration.
  #   1) In addition to `worker` tasks, a `chief` task type is use;
  #      in this case, this function should be modified to
  #      `return task_type == 'chief'`.
  #   2) Only `worker` task type is used; in this case, worker 0 is
  #      regarded as the chief. The implementation demonstrated here
  #      is for this case.
  # For the purpose of this Colab section, the `task_type is None` case
  # is added because it is effectively run with only a single worker.
  return (task_type == 'worker' and task_id == 0) or task_type is None

def _get_temp_dir(dirpath, task_id):
  base_dirpath = 'workertemp_' + str(task_id)
  temp_dir = os.path.join(dirpath, base_dirpath)
  tf.io.gfile.makedirs(temp_dir)
  return temp_dir

def write_filepath(filepath, task_type, task_id):
  dirpath = os.path.dirname(filepath)
  base = os.path.basename(filepath)
  if not _is_chief(task_type, task_id):
    dirpath = _get_temp_dir(dirpath, task_id)
  return os.path.join(dirpath, base)

task_type, task_id = (strategy.cluster_resolver.task_type,
                      strategy.cluster_resolver.task_id)
write_model_path = write_filepath(model_path, task_type, task_id)

이제 저장할 준비가 되었습니다.

In [ ]:
multi_worker_model.save(write_model_path)

위에서 설명한 것처럼 나중에 모델은 수석 작업자가 저장된 경로에서만 로드되어야 하므로 수석이 아닌 작업자가 저장한 임시 경로는 제거하겠습니다.

In [ ]:
if not _is_chief(task_type, task_id):
  tf.io.gfile.rmtree(os.path.dirname(write_model_path))

이제 로드할 때 편리한 `tf.keras.models.load_model` API를 사용하여 추가 작업을 진행해 보겠습니다.

여기에서는 단일 작업자만 사용하여 훈련을 로드하고 계속한다고 가정합니다. 이 경우 다른 `strategy.scope()` 내에서 `tf.keras.models.load_model`을 호출하지 않습니다(앞서 정의한 바와 같이 `strategy = tf.distribute.MultiWorkerMirroredStrategy()`임에 유의):

In [ ]:
loaded_model = tf.keras.models.load_model(model_path)

# Now that the model is restored, and can continue with the training.
loaded_model.fit(single_worker_dataset, epochs=2, steps_per_epoch=20)

### 체크포인트 저장 및 복원

한편, 체크포인트를 사용하면 전체 모델을 저장하지 않고도 모델의 가중치를 저장하고 복원할 수 있습니다.

여기에서는 모델을 추적하는 하나의 `tf.train.Checkpoint`를 만들고, 최신 체크포인트만 보존되도록 이를 `tf.train.CheckpointManager`로 관리합니다.

In [ ]:
checkpoint_dir = '/tmp/ckpt'

checkpoint = tf.train.Checkpoint(model=multi_worker_model)
write_checkpoint_dir = write_filepath(checkpoint_dir, task_type, task_id)
checkpoint_manager = tf.train.CheckpointManager(
    checkpoint, directory=write_checkpoint_dir, max_to_keep=1)

`CheckpointManager`가 설정되면 이제 수석이 아닌 작업자가 저장한 체크포인트를 저장하고 제거할 준비가 된 것입니다.

In [ ]:
checkpoint_manager.save()
if not _is_chief(task_type, task_id):
  tf.io.gfile.rmtree(write_checkpoint_dir)

이제 모델을 복원해야 할 때 편리한 `tf.train.latest_checkpoint` 함수를 사용하여 저장된 최신 체크포인트를 찾을 수 있습니다. 체크포인트를 복원한 후 훈련을 계속할 수 있습니다.

In [ ]:
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint.restore(latest_checkpoint)
multi_worker_model.fit(multi_worker_dataset, epochs=2, steps_per_epoch=20)

#### BackupAndRestore 콜백

`tf.keras.callbacks.BackupAndRestore` 콜백은 임시 체크포인트 파일의 모델과 현재 epoch 수를 `BackupAndRestore`에 대한 `backup_dir` 인수 아래의 임시 체크포인트 파일에 백업하여 내결함성 기능을 제공합니다. 이 작업은 각 epoch가 끝날 때 수행됩니다.

작업이 중단되고 다시 시작되면 콜백이 마지막 체크포인트를 복원하고 중단된 epoch의 시작 부분부터 훈련이 계속됩니다. 중단되기 전에 완료되지 않은 epoch에서 이미 수행된 부분적 훈련은 폐기되므로 최종 모델 상태에 영향을 주지 않습니다.

이를 사용하려면 `Model.fit` 호출에서 `tf.keras.callbacks.BackupAndRestore`의 인스턴스를 제공하세요.

`MultiWorkerMirroredStrategy`를 사용하면 작업자가 중단되었을 때 중단된 작업자가 다시 시작될 때까지 전체 클러스터가 일시 중지됩니다. 다른 작업자도 다시 시작되고 중단된 작업자가 클러스터에 다시 참여합니다. 그런 다음 모든 작업자는 이전에 저장한 체크포인트 파일을 읽고 이전 상태를 가져오므로 클러스터가 다시 동기화될 수 있습니다. 그런 다음 훈련이 계속됩니다.

`BackupAndRestore` 콜백은 `CheckpointManager`를 사용하여 훈련 상태를 저장하고 복원하는데, 이 때 최신 체크포인트와 함께 기존 체크포인트를 추적하는 체크포인트라는 파일이 생성됩니다. 이러한 이유로 이름 충돌을 피하기 위해 `backup_dir`는 다른 체크포인트를 저장하는 데 재사용되지 않아야 합니다.

Currently, the `BackupAndRestore` callback supports single-worker training with no strategy—`MirroredStrategy`—and multi-worker training with `MultiWorkerMirroredStrategy`.

Below are two examples for both multi-worker training and single-worker training:

In [ ]:
# Multi-worker training with `MultiWorkerMirroredStrategy`
# and the `BackupAndRestore` callback.

callbacks = [tf.keras.callbacks.BackupAndRestore(backup_dir='/tmp/backup')]
with strategy.scope():
  multi_worker_model = mnist_setup.build_and_compile_cnn_model()
multi_worker_model.fit(multi_worker_dataset,
                       epochs=3,
                       steps_per_epoch=70,
                       callbacks=callbacks)

`BackupAndRestore`에서 지정한 `backup_dir`의 디렉터리를 살펴보면 일시적으로 생성된 일부 체크포인트 파일을 발견할 수 있습니다. 이러한 파일은 이전에 손실된 인스턴스를 복구하는 데 필요하며 훈련이 성공적으로 종료되면 `Model.fit` 마지막에 라이브러리에서 제거됩니다.

참고: 현재, `BackupAndRestore` 콜백은 즉시 실행 모드만 지원합니다. 그래프 모드에서는 위에서 언급한 [Save/Restore Model](#model_saving_and_loading)을 사용하고 `Model.fit`에서 `initial_epoch`를 제공하는 방법을 고려하세요.

## Additional resources

1. [TensorFlow에서 분산 훈련하기](https://www.tensorflow.org/guide/distributed_training) 가이드는 사용 가능한 분산 전략을 간략히 소개합니다.
2. [Keras 및 MultiWorkerMirroredStrategy를 이용한 사용자 지정 훈련 루프](multi_worker_with_ctl.ipynb) 튜토리얼은 Keras 및 사용자 지정 훈련 루프와 함께 `MultiWorkerMirroredStrategy`를 이용하는 방법을 보여줍니다.
3. 여러 분산 전략을 실행하도록 구성할 수 있는 [공식 모델](https://github.com/tensorflow/models/tree/master/official)을 확인해 보세요.
4. [tf.function으로 성능 향상](../../guide/function.ipynb) 가이드는 TensorFlow 모델의 성능을 최적화하는 데 사용할 수 있는 [TensorFlow 프로파일러](../../guide/profiler.md)와 같은 다른 전략 및 도구에 대한 정보를 제공합니다.